# 📊 Complete Trading System Documentation

## 🎯 Overview

This is your **comprehensive automated paper trading system** that combines:
- **Bayesian Machine Learning** for probabilistic forecasting
- **Gaussian Processes** for non-linear pattern recognition  
- **Technical Analysis** (RSI) for market sentiment
- **Real-time Alpaca API** for paper trading execution

---

## 🧠 The 5 W's + H

### **WHO** uses this system?
- **You** - retail trader using algorithmic strategies
- **Target**: Any trader wanting quantitative, probability-based trading decisions
- **Skill Level**: Intermediate Python + understanding of basic statistics

### **WHAT** does it do?
1. **Downloads** historical stock data (Yahoo Finance)
2. **Analyzes** price movements using 3 independent models:
   - Bayesian Linear Regression
   - Gaussian Process Regression
   - RSI Technical Indicator
3. **Generates** trading signals (BUY/SELL/HOLD) with confidence scores
4. **Executes** paper trades via Alpaca API with automated bracket orders (take-profit + stop-loss)
5. **Manages** positions with risk controls

### **WHEN** to use it?
- **During market hours** (DAY orders execute when market opens if submitted after hours)
- **Daily/Weekly**: Run scheduled scans of your universe
- **Before trading**: Generate signals and review recommendations
- **Real-time**: Monitor positions and adjust thresholds

### **WHERE** does it connect?
- **Data Source**: Yahoo Finance (yfinance library) - free historical data
- **Broker**: Alpaca Markets API (paper trading account - no real money)
- **Execution**: Cloud-ready (can run on local machine or server)

### **WHY** this approach?
✅ **Probabilistic**: Every prediction includes uncertainty (confidence intervals)  
✅ **Multi-model**: Combines 3 independent approaches = more robust  
✅ **Automated**: Removes emotional decision-making  
✅ **Risk-managed**: Bracket orders = automatic profit-taking + loss-limiting  
✅ **Paper trading**: Test strategies without risking real capital  

### **HOW** does it work?
```
1. DATA COLLECTION → yfinance downloads historical prices
2. FEATURE ENGINEERING → Creates lagged returns, volatility, RSI
3. MODEL 1: Bayesian Linear Regression → Predicts next return using weighted averages
4. MODEL 2: Gaussian Process → Captures non-linear patterns in price history
5. MODEL 3: RSI Analysis → Bayesian interpretation of oversold/overbought levels
6. ENSEMBLE → Combines all 3 models into weighted consensus
7. SIGNAL GENERATION → BUY if bullish consensus, SELL if bearish, HOLD otherwise
8. ORDER EXECUTION → Submits bracket orders to Alpaca API
9. POSITION MONITORING → Tracks TP/SL and manages portfolio
```

---

## 🔧 Core Functions Deep Dive

---

### 1️⃣ **`unified_bayesian_gp_forecast(ticker)`** - The Brain 🧠

**Location**: `functions/trading_functions.py`

**What it does**:
- Downloads 200 days of historical data for a stock
- Trains 3 independent forecasting models
- Combines them into ensemble prediction
- Generates trading signal with confidence score

**How it works**:

```python
# Example usage:
result = unified_bayesian_gp_forecast("AAPL")
# Returns dict with:
# - bayesian/gp/ensemble forecasts
# - confidence levels
# - final_signal: "BUY", "SELL", or "HOLD"
# - RSI analysis
```

**The 3 Models Inside**:

#### 📘 **Model 1: Bayesian Linear Regression**
- **Input Features**: Last 5 days' returns + volatility + RSI
- **Method**: Weighted average of past patterns
- **Output**: Expected return + uncertainty range
- **Strength**: Fast, interpretable, handles noise well
- **When it's useful**: Trending markets with clear momentum

#### 📙 **Model 2: Gaussian Process**
- **Input Features**: Last 10 days' returns only
- **Method**: Non-parametric - learns smooth functions from data
- **Output**: Expected return + uncertainty range  
- **Strength**: Captures non-linear patterns, adapts to complexity
- **When it's useful**: Choppy markets with regime changes

#### 📕 **Model 3: RSI (Bayesian Interpretation)**
- **Input**: 14-day RSI value
- **Method**: Bayesian probability of BUY/SELL/HOLD
- **Output**: Signal + probability distribution
- **Strength**: Classic technical analysis with probabilistic twist
- **When it's useful**: Mean-reversion plays, contrarian signals

**Ensemble Logic**:
```python
# Models are weighted by inverse uncertainty
# More confident model = higher weight
ensemble_forecast = (bayesian_weight * bayesian_pred + gp_weight * gp_pred) / total_weight

# Final signal requires 2+ models agreeing:
# STRONG BUY: 3+ bullish signals
# BUY: 2 bullish signals
# STRONG SELL: 3+ bearish signals
# SELL: 2 bearish signals  
# HOLD: No consensus
```

**Returns**: Dictionary with all forecasts, uncertainties, and final trading recommendation

---

### 2️⃣ **`extract_signal_confidence(result)`** - Signal Parser 🔍

**Location**: `paper_trading.ipynb` (helper function)

**What it does**:
- Extracts the BUY/SELL/HOLD signal from model output
- Extracts confidence score (0.0 to 1.0)
- Handles different output formats

**When to use**: After running any forecasting function to get standardized signal

**Example**:
```python
result = unified_bayesian_gp_forecast("TSLA")
signal, confidence = extract_signal_confidence(result)
# signal = "buy", confidence = 0.75
```

---

### 3️⃣ **`fetch_latest_prices(symbols)`** - Live Price Fetcher 💰

**Location**: `paper_trading.ipynb`

**What it does**:
- Gets current/latest price for list of stocks
- Tries multiple Alpaca endpoints with fallbacks:
  1. Latest trade (real-time)
  2. Latest bar close (most recent candle)
  3. Last minute bar (1-min ago)
  4. Prior day close (if market closed)

**When to use**: Before calculating position sizes or checking current value

**Example**:
```python
prices = fetch_latest_prices(["AAPL", "MSFT", "GOOGL"])
# Returns: {"AAPL": 185.32, "MSFT": 378.91, "GOOGL": 142.17}
```

---

### 4️⃣ **`preview_signal(symbol)`** - Quick Preview 👀

**Location**: `paper_trading.ipynb`

**What it does**:
- Runs full forecast for one stock
- Prints human-readable summary
- Shows expected move, z-score, probability of upside
- Calculates TP/SL prices
- Estimates position size

**When to use**: Before running full batch - test individual stocks

**Example Output**:
```
Symbol: AAPL
Last: $185.32
Expected move: 1.2% (~$2.22)
Z-score: 0.85, P(upside): 80.2%
Signal: BUY @ 75%
TP: $192.73, SL: $181.61
Sim qty (notional $100): 0.5395 shares
```

---

### 5️⃣ **`summarize_universe(symbols)`** - Batch Scanner 📊

**Location**: `paper_trading.ipynb`

**What it does**:
- Runs forecasts for ALL stocks in universe
- Suppresses plots/output for speed
- Returns pandas DataFrame with all results
- Sorts by signal quality (BUY → HOLD → SELL, then by confidence)

**When to use**: Daily scan to identify best opportunities

**Example**:
```python
df = summarize_universe(UNIVERSE)
# Returns DataFrame with columns:
# symbol, last, signal, confidence, forecast_pct, std_pct, z, p_up, tp, sl, sim_qty
```

---

### 6️⃣ **`run_once()`** - The Trading Engine ⚙️

**Location**: `paper_trading.ipynb`

**What it does** (step-by-step):

1. **Check market status** (open/closed)
2. **Fetch prices** for all universe stocks
3. **Run signals** for each stock (calls `unified_bayesian_gp_forecast`)
4. **Filter candidates**:
   - Primary: explicit BUY signal with confidence ≥ MIN_CONF
   - Fallback: ensemble shows positive expected move with:
     - P(upside) ≥ MIN_PROB_UP
     - Z-score ≥ MIN_Z
     - Expected USD move ≥ MIN_USD_MOVE
   - Relax: If nothing passes, take top picks by probability
5. **Rank by confidence** (best opportunities first)
6. **Size positions**:
   - If USE_NOTIONAL=True: Split $NOTIONAL_CAPITAL equally
   - If False: Use 95% of account buying_power
7. **Check for existing orders** (skip if already pending)
8. **Submit bracket orders** to Alpaca:
   - Market BUY order
   - Take-profit limit @ +TP_PCT%
   - Stop-loss @ -SL_PCT%
9. **Return results** (order IDs or DRY_RUN confirmation)

**When to use**: This is your main execution function - run daily or on-demand

**Configuration Variables**:
```python
DRY_RUN = True           # Set False to execute real orders
MIN_CONF = 0.65          # Minimum confidence for explicit signals
MAX_ORDERS_PER_RUN = 20  # Max positions per execution
USE_NOTIONAL = True      # Use fake $100 vs real account balance
NOTIONAL_CAPITAL = 100   # Fake capital for testing
TP_PCT = 0.04            # Take profit at +4%
SL_PCT = 0.02            # Stop loss at -2%
```

**Advanced Filtering** (ensemble fallback):
```python
MIN_PROB_UP = 0.60      # Minimum upside probability
MIN_Z = 0.20            # Minimum z-score (edge)
MIN_USD_MOVE = 0.05     # Minimum expected dollar move
TARGET_MIN_PICKS = 3    # Relaxed mode: ensure at least 3 picks
```

---

## 🎬 Real-World Usage Workflow

---

### **Scenario 1: Daily Morning Routine** ☕

**Goal**: Scan market and place new trades

```python
# 1. Set to paper trading mode
DRY_RUN = False  # Actually execute orders
USE_NOTIONAL = True
NOTIONAL_CAPITAL = 100

# 2. Define your watch list
UNIVERSE = ["SPY", "AAPL", "MSFT", "NVDA", "GOOGL", "TSLA", "AMZN", "META"]

# 3. Run the scanner (see what's hot)
df = summarize_universe(UNIVERSE)
print(tabulate(df, headers="keys", tablefmt="github"))

# 4. Review top candidates
# Look for: BUY signals with confidence > 70%, positive z-scores, reasonable TP/SL

# 5. Execute trades
results = run_once()
# This submits bracket orders for best opportunities
```

**What happens**:
- Scans 8 stocks
- Identifies 2-3 with strong BUY signals
- Places $100/3 = $33.33 per position
- Each order has automatic TP (+4%) and SL (-2%)

---

### **Scenario 2: Test Single Stock Before Trading** 🧪

**Goal**: Deep dive into one ticker before committing

```python
# Preview single stock with full details
preview_signal("AAPL")

# See the full analysis with plots
result = unified_bayesian_gp_forecast("AAPL")

# Check the ensemble recommendation
print(f"Signal: {result['final_signal']}")
print(f"Confidence: {result['confidence']:.1%}")
print(f"Expected return: {result['ensemble']['forecast']:.2%}")
```

**Use this when**:
- You want to understand WHY a signal was generated
- Testing new stocks before adding to universe
- Debugging unexpected signals

---

### **Scenario 3: Tune Your Thresholds** ⚙️

**Goal**: Adjust strategy aggressiveness

```python
# CONSERVATIVE SETUP (fewer, higher-confidence trades)
MIN_CONF = 0.75          # Only very confident signals
MIN_PROB_UP = 0.65       # Require 65% upside probability
MIN_Z = 0.40             # Strong statistical edge
TP_PCT = 0.06            # Take profit at +6%
SL_PCT = 0.03            # Stop loss at -3%
MAX_ORDERS_PER_RUN = 5   # Limit positions

# AGGRESSIVE SETUP (more trades, accept more risk)
MIN_CONF = 0.60          # Lower confidence threshold
MIN_PROB_UP = 0.55       # Accept 55% upside probability
MIN_Z = 0.15             # Smaller edge acceptable
TP_PCT = 0.03            # Quick profit at +3%
SL_PCT = 0.015           # Tight stop at -1.5%
MAX_ORDERS_PER_RUN = 15  # More positions

# Then run:
run_once()
```

---

### **Scenario 4: Monitor Existing Positions** 📈

**Goal**: Check your paper trading account

```python
# Get all open positions
positions = trading.get_all_positions()

# Display nicely
from tabulate import tabulate
table_data = []
for pos in positions:
    table_data.append([
        pos.symbol,
        pos.qty,
        f"${pos.avg_entry_price}",
        f"${pos.current_price}",
        f"${pos.unrealized_pl}"
    ])

print(tabulate(table_data, 
               headers=["Symbol", "Qty", "Entry", "Current", "P&L"],
               tablefmt="fancy_grid"))

# Check pending orders
orders = trading.get_orders(status=QueryOrderStatus.OPEN)
for order in orders:
    print(f"{order.symbol}: {order.side} {order.qty} @ {order.order_type}")
```

---

### **Scenario 5: Backtest a Strategy** 📊

**Goal**: Test historical performance (not live trading)

**Use these functions**:

```python
# 1. RSI Backtest (from trading_functions.py)
# This uses the Strategy class with RSI signals

df = yf.download("AAPL", period="1y", interval="1d")
df = rsi_calculator(df)

strategy = Strategy(df, starting_balance=10000)
results = strategy.run()

# See trade history
print(results)
print(f"Final P&L: ${results['pnl'].iloc[-1] - 10000:.2f}")

# 2. Forecast Performance Analysis
# Run forecast_next_day_return to see Bayesian predictions vs actual

forecast_next_day_return("AAPL", period="200d")
# This plots predictions vs reality + confidence intervals
```

---

### **Scenario 6: Volatility Regime Analysis** 📉

**Goal**: Understand how stock behaves in different market conditions

```python
# Analyze how AAPL moves in high vs low volatility
analyze_volatility_regimes(
    ticker_list=["AAPL", "TSLA", "SPY"],
    period="1y",
    interval="1d",
    window=20  # 20-day rolling volatility
)

# Output shows:
# P(up | high volatility): 52.3%
# P(up | low volatility): 58.7%
# → Suggests AAPL performs better in calm markets
```

**Use this to**:
- Identify mean-reverting vs trending stocks
- Adjust position sizing based on current regime
- Filter universe (avoid stocks with low P(up) in current regime)

---

## 🔗 How Functions Connect to Paper Trading

---

### **Data Flow Architecture**

```
┌─────────────────────────────────────────────────────────┐
│           YAHOO FINANCE (yfinance)                      │
│           Historical OHLCV Data                         │
└────────────────────┬────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────┐
│     FEATURE ENGINEERING (trading_functions.py)          │
│     - rsi_calculator(): Add RSI column                  │
│     - Lagged returns (Return_lag1, Return_lag2...)      │
│     - Rolling volatility                                │
└────────────────────┬────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────┐
│     FORECASTING MODELS (unified_bayesian_gp_forecast)   │
│                                                         │
│     ┌───────────────────────────────────────┐          │
│     │  Bayesian Linear Regression           │          │
│     │  update_beliefs_with_data()           │          │
│     └───────────────────────────────────────┘          │
│                                                         │
│     ┌───────────────────────────────────────┐          │
│     │  Gaussian Process                     │          │
│     │  gp_predict() + rbf_kernel()          │          │
│     └───────────────────────────────────────┘          │
│                                                         │
│     ┌───────────────────────────────────────┐          │
│     │  RSI Signal                           │          │
│     │  bayesian_rsi_signal()                │          │
│     └───────────────────────────────────────┘          │
│                                                         │
│     → Ensemble weighted by uncertainty                 │
│     → Final signal: BUY/SELL/HOLD + confidence         │
└────────────────────┬────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────┐
│     SIGNAL PROCESSING (paper_trading.ipynb)             │
│     - extract_signal_confidence()                       │
│     - Filter by MIN_CONF, MIN_PROB_UP, MIN_Z            │
│     - Rank candidates by confidence                     │
└────────────────────┬────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────┐
│     PRICE FETCHING (Alpaca Data API)                    │
│     - fetch_latest_prices()                             │
│     - StockHistoricalDataClient                         │
└────────────────────┬────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────┐
│     POSITION SIZING                                     │
│     - If USE_NOTIONAL: NOTIONAL_CAPITAL / num_picks     │
│     - Else: account.buying_power * 0.95 / num_picks     │
│     - qty = dollars_per_symbol / current_price          │
└────────────────────┬────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────┐
│     RISK MANAGEMENT                                     │
│     - TP price = entry_price * (1 + TP_PCT)             │
│     - SL price = entry_price * (1 - SL_PCT)             │
│     - has_open_order() check to avoid duplicates        │
└────────────────────┬────────────────────────────────────┘
                     │
                     ▼
┌─────────────────────────────────────────────────────────┐
│     ORDER EXECUTION (Alpaca Trading API)                │
│     - TradingClient.submit_order()                      │
│     - MarketOrderRequest (BUY side)                     │
│     - OrderClass.BRACKET                                │
│     - TakeProfitRequest + StopLossRequest               │
│     - TimeInForce.DAY                                   │
└─────────────────────────────────────────────────────────┘
```

---

### **Function Mapping to Paper Trading Flow**

| **Stage** | **Function Used** | **Located In** | **Purpose** |
|-----------|-------------------|----------------|-------------|
| **1. Data Collection** | `yf.download()` | Import (yfinance) | Download historical prices |
| **2. Feature Engineering** | `rsi_calculator()` | `trading_functions.py` | Calculate RSI indicator |
| **3. Forecasting** | `unified_bayesian_gp_forecast()` | `trading_functions.py` | Generate prediction + signal |
| **4. Signal Extraction** | `extract_signal_confidence()` | `paper_trading.ipynb` | Parse model output |
| **5. Price Fetching** | `fetch_latest_prices()` | `paper_trading.ipynb` | Get current prices |
| **6. Candidate Screening** | `run_once()` filtering logic | `paper_trading.ipynb` | Apply MIN_CONF, MIN_Z filters |
| **7. Position Sizing** | `run_once()` sizing logic | `paper_trading.ipynb` | Calculate qty per symbol |
| **8. Duplicate Check** | `has_open_order()` | `paper_trading.ipynb` | Avoid double-ordering |
| **9. Order Submission** | `trading.submit_order()` | Alpaca SDK | Execute bracket order |
| **10. Monitoring** | `trading.get_all_positions()` | Alpaca SDK | Check open positions |

---

## 💡 Advanced Tips & Best Practices

---

### **1. Optimal Threshold Settings** 🎯

**For day trading** (quick in/out):
```python
MIN_CONF = 0.70          # Higher confidence required
TP_PCT = 0.02            # Take profit at +2% 
SL_PCT = 0.01            # Tight stop at -1%
MAX_ORDERS_PER_RUN = 3   # Focus on best 3 only
```

**For swing trading** (hold days/weeks):
```python
MIN_CONF = 0.65          # Moderate confidence OK
TP_PCT = 0.08            # Take profit at +8%
SL_PCT = 0.04            # Wider stop at -4%
MAX_ORDERS_PER_RUN = 10  # Diversify across 10 stocks
```

**For mean reversion** (RSI-focused):
```python
# In unified_bayesian_gp_forecast, RSI gets more weight
# Look for:
# - BUY when RSI < 30 AND ensemble forecast > 0
# - SELL when RSI > 70 AND ensemble forecast < 0
```

---

### **2. Interpreting Model Outputs** 📊

**Bayesian Linear Regression**:
- **forecast**: Expected return (e.g., 0.012 = +1.2%)
- **std**: Uncertainty (e.g., 0.005 = ±0.5%)
- **95% CI**: forecast ± 2*std
- ✅ **Trust when**: Low std (< 0.01), trending market
- ❌ **Skeptical when**: High std (> 0.03), choppy market

**Gaussian Process**:
- **forecast**: Expected return (non-linear pattern)
- **std**: Uncertainty from kernel smoothness
- ✅ **Trust when**: Recent data shows clear patterns
- ❌ **Skeptical when**: Market regime just changed

**Ensemble**:
- Weighted average of both models
- Lower std = higher weight in ensemble
- **Final signal requires 2+ models agreeing**

---

### **3. Risk Management Rules** 🛡️

**Never risk more than 2% per trade**:
```python
# If you had $10,000 real capital:
RISK_PER_TRADE = 0.02  # 2%
CAPITAL = 10000
SL_PCT = 0.02          # 2% stop loss

max_loss_per_trade = CAPITAL * RISK_PER_TRADE  # $200
position_size = max_loss_per_trade / SL_PCT     # $10,000
# This is your max position size
```

**Portfolio heat check**:
```python
# Don't have more than 30% of capital at risk simultaneously
positions = trading.get_all_positions()
total_market_value = sum(float(p.market_value) for p in positions)
account = trading.get_account()
portfolio_value = float(account.portfolio_value)

heat = total_market_value / portfolio_value
if heat > 0.30:
    print("⚠️ Too much exposure! Reduce positions.")
```

---

### **4. When to Override Signals** 🚨

**Ignore BUY signals if**:
- SPY (market) is in downtrend
- VIX > 30 (high volatility/fear)
- Stock just reported bad earnings
- Already have 3+ positions in same sector

**Force SELL if**:
- Holding period > 30 days with no TP hit
- New fundamental news changes thesis
- Stop loss keeps getting hit (strategy not working for this stock)

---

### **5. Monitoring & Debugging** 🔍

**If no signals generated**:
```python
# Check thresholds are not too strict
print(f"MIN_CONF: {MIN_CONF}")
print(f"MIN_PROB_UP: {MIN_PROB_UP}")
print(f"MIN_Z: {MIN_Z}")

# Try running preview_signal on each stock manually
for sym in UNIVERSE:
    preview_signal(sym)
```

**If signals look wrong**:
```python
# Check data quality
df = yf.download("AAPL", period="200d")
print(df.tail(10))  # Last 10 days - any gaps?

# Re-run with plots enabled (see visual patterns)
result = unified_bayesian_gp_forecast("AAPL")
# Check if plots show reasonable forecasts
```

**If orders fail**:
```python
# Check API keys
print("API_KEY loaded:", bool(API_KEY))

# Check buying power
account = trading.get_account()
print(f"Buying power: ${account.buying_power}")

# Check market is open
clock = trading.get_clock()
print(f"Market open: {clock.is_open}")
```

---

### **6. Transition from Paper → Real Money** 💰

**Before going live**:

1. ✅ **Run paper trading for 30+ days**
2. ✅ **Track win rate** (should be > 50% for profitable strategy)
3. ✅ **Calculate Sharpe ratio** (returns / volatility > 1.0 is good)
4. ✅ **Test with real market hours** (some strategies fail in live conditions)
5. ✅ **Start with 10% of intended capital** (e.g., if planning $10k, start with $1k)

**When ready**:
```python
# Change ONE line:
trading = TradingClient(API_KEY, SECRET_KEY, paper=False)  # Live trading!

# Keep everything else the same
# Start with small positions
USE_NOTIONAL = False  # Use real buying power
MAX_ORDERS_PER_RUN = 1  # Start with 1 position at a time
```

---

### **7. Performance Tracking** 📈

**Create a trade log**:
```python
import json
from datetime import datetime

trade_log = []

# After each run_once()
for sym, order_id in results:
    trade_log.append({
        "date": str(datetime.now()),
        "symbol": sym,
        "order_id": order_id,
        "entry_price": prices[sym],
        "signal_confidence": confidence,
        "tp": tp_price,
        "sl": sl_price
    })

# Save to file
with open("trade_log.json", "w") as f:
    json.dump(trade_log, f, indent=2)
```

**Analyze results**:
```python
import pandas as pd

# Load log
with open("trade_log.json") as f:
    log = json.load(f)

df = pd.DataFrame(log)

# Metrics
closed_positions = [p for p in positions if p.status == "closed"]
wins = [p for p in closed_positions if p.profit > 0]
losses = [p for p in closed_positions if p.profit < 0]

win_rate = len(wins) / len(closed_positions) if closed_positions else 0
avg_win = np.mean([p.profit for p in wins]) if wins else 0
avg_loss = np.mean([p.profit for p in losses]) if losses else 0

print(f"Win Rate: {win_rate:.1%}")
print(f"Avg Win: ${avg_win:.2f}")
print(f"Avg Loss: ${avg_loss:.2f}")
print(f"Profit Factor: {abs(avg_win / avg_loss) if avg_loss != 0 else 'N/A'}")
```

---

## 📚 Function Reference Quick Guide

---

### **From `trading_functions.py`**

| Function | Use Case | Returns |
|----------|----------|---------|
| `rsi_calculator(df)` | Add RSI column to DataFrame | DataFrame with `rsi_14` column |
| `bayesian_rsi_signal(rsi_value)` | Get BUY/SELL/HOLD from RSI | (signal, probabilities) |
| `update_beliefs_with_data(X, y, ...)` | Bayesian regression update | (posterior_mean, posterior_cov) |
| `rbf_kernel(X1, X2, ...)` | Compute similarity for GP | Kernel matrix |
| `gp_predict(X_train, y_train, X_test)` | Gaussian Process forecast | (pred_mean, pred_var) |
| `unified_bayesian_gp_forecast(ticker)` | **MAIN FORECASTING FUNCTION** | Dict with signals & forecasts |
| `forecast_next_day_return(ticker)` | Simple Bayesian forecast | Prints prediction + advice |
| `analyze_volatility_regimes(tickers, ...)` | Volatility regime analysis | Prints P(up\|high vol) stats |
| `display_portfolio_ohlcv_heads(tickers, ...)` | Show OHLCV data for stocks | Displays DataFrames |

---

### **From `paper_trading.ipynb`**

| Function | Use Case | Returns |
|----------|----------|---------|
| `extract_signal_confidence(result)` | Parse model output | (signal, confidence) |
| `fetch_latest_prices(symbols)` | Get current prices | Dict {symbol: price} |
| `has_open_order(symbol, side)` | Check for pending orders | Boolean |
| `has_long_position(symbol)` | Check if holding stock | Boolean |
| `preview_signal(symbol)` | Quick single-stock preview | Prints summary + full result |
| `summarize_universe(symbols)` | Batch scan all stocks | DataFrame with all results |
| `suppress_output()` | Context manager to hide prints | None (context manager) |
| `suppress_plots()` | Context manager to hide plots | None (context manager) |
| `run_once()` | **MAIN EXECUTION FUNCTION** | List of (symbol, order_id) tuples |

---

### **From Alpaca SDK**

| Function | Use Case | Returns |
|----------|----------|---------|
| `trading.get_account()` | Get account info | Account object |
| `trading.get_all_positions()` | Get open positions | List of Position objects |
| `trading.get_orders(filter=...)` | Get orders | List of Order objects |
| `trading.submit_order(order_request)` | Submit new order | Order object |
| `trading.get_clock()` | Check market status | Clock object |
| `data.get_stock_latest_trade(request)` | Get latest trade price | Trade object |
| `data.get_stock_bars(request)` | Get historical bars | Bars object |

---

## 🚀 Complete Example: From Zero to Trade

---

### **Step-by-Step Tutorial**

This shows EXACTLY how to use all your functions in a real paper trading session.

---

#### **Setup Phase** (Run Once)

In [ ]:
# ═══════════════════════════════════════════════════════════
# STEP 1: IMPORTS & SETUP (One-time setup)
# ═══════════════════════════════════════════════════════════

from dotenv import load_dotenv
import os, time
from tabulate import tabulate
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, GetOrdersRequest, TakeProfitRequest, StopLossRequest
from alpaca.trading.enums import OrderSide, TimeInForce, QueryOrderStatus, OrderClass
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime, timezone
from pathlib import Path
import sys
import pandas as pd

# Load API credentials
load_dotenv(dotenv_path="./.env")
API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")

# Initialize Alpaca clients
trading = TradingClient(API_KEY, SECRET_KEY, paper=True)  # Paper trading
data = StockHistoricalDataClient(API_KEY, SECRET_KEY)

# Import your custom forecasting function
project_root = Path(r"c:\Users\Windows User\My_folder\Finance_project")
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))
from functions.trading_functions import unified_bayesian_gp_forecast

print("✅ Setup complete!")

In [ ]:
# ═══════════════════════════════════════════════════════════
# STEP 2: CONFIGURE STRATEGY PARAMETERS
# ═══════════════════════════════════════════════════════════

# Trading mode
DRY_RUN = True           # Set False to execute real paper trades
USE_NOTIONAL = True      # Use fake capital vs real buying power
NOTIONAL_CAPITAL = 100   # Fake $100 for testing

# Signal filtering
MIN_CONF = 0.65          # Minimum confidence for BUY signals
MIN_PROB_UP = 0.60       # Minimum probability of upside (ensemble fallback)
MIN_Z = 0.20             # Minimum z-score (statistical edge)
MIN_USD_MOVE = 0.05      # Minimum expected dollar move

# Risk management
TP_PCT = 0.04            # Take profit at +4%
SL_PCT = 0.02            # Stop loss at -2%

# Position limits
MAX_ORDERS_PER_RUN = 20  # Max trades per execution

# Relaxed selection (if strict filters produce nothing)
ALT_SELECTION = True     # Enable ensemble fallback
RELAX_SELECTION = True   # Relax thresholds if needed
TARGET_MIN_PICKS = 3     # Minimum picks in relaxed mode
DEBUG_SELECTION = True   # Print selection logic

# Define your universe
UNIVERSE = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "NVDA", "TSLA", "JPM", "V", "UNH", "SPY"]

print("📋 Configuration:")
print(f"   Universe: {len(UNIVERSE)} stocks")
print(f"   DRY_RUN: {DRY_RUN}")
print(f"   Capital: ${NOTIONAL_CAPITAL}")
print(f"   TP/SL: {TP_PCT:.1%}/{SL_PCT:.1%}")

In [ ]:
# ═══════════════════════════════════════════════════════════
# STEP 3: TEST SINGLE STOCK (Preview before trading)
# ═══════════════════════════════════════════════════════════

# Test the forecasting function on one stock
test_symbol = "AAPL"
print(f"🧪 Testing forecasting function on {test_symbol}...")

result = unified_bayesian_gp_forecast(test_symbol)

# This will show:
# - 6 plots showing model performance
# - Printed summary with signals and confidence
# - Returns a dictionary with all the data

print("\n📊 Result structure:")
if result:
    print(f"   Signal: {result['final_signal']}")
    print(f"   Confidence: {result['final_confidence']:.1%}")
    print(f"   Ensemble forecast: {result['ensemble']['forecast']:.2%}")
else:
    print("   ❌ No result (insufficient data)")

In [ ]:
# ═══════════════════════════════════════════════════════════
# STEP 4: SCAN ENTIRE UNIVERSE (Find best opportunities)
# ═══════════════════════════════════════════════════════════

# This suppresses plots/prints for batch processing
import io
from contextlib import contextmanager, redirect_stdout, redirect_stderr
import matplotlib.pyplot as plt

@contextmanager
def suppress_output(enabled=True):
    if not enabled:
        yield
        return
    buf = io.StringIO()
    with redirect_stdout(buf), redirect_stderr(buf):
        yield

@contextmanager
def suppress_plots(enabled=True):
    if not enabled:
        yield
        return
    _show = plt.show
    try:
        plt.show = lambda *a, **k: None
        yield
    finally:
        plt.show = _show

# Scan function
def summarize_universe(symbols):
    """Scan all symbols and return DataFrame with signals"""
    from functions.trading_functions import unified_bayesian_gp_forecast
    import math
    
    # Fetch prices
    from paper_trading import fetch_latest_prices  # Use your existing function
    prices = fetch_latest_prices(symbols)
    
    rows = []
    for s in symbols:
        print(f"Scanning {s}...", end=" ")
        with suppress_output(True), suppress_plots(True):
            raw = unified_bayesian_gp_forecast(s)
        
        # Extract signal
        sig = raw.get('final_signal') if raw else None
        conf = raw.get('final_confidence') if raw else None
        
        # Extract ensemble stats
        if raw and 'ensemble' in raw:
            ens = raw['ensemble']
            f_pct = ens.get('forecast')
            std_pct = ens.get('std')
            z = f_pct / std_pct if (f_pct and std_pct and std_pct > 0) else None
            p_up = 0.5 * (1.0 + math.erf(z / math.sqrt(2.0))) if z else None
        else:
            f_pct = std_pct = z = p_up = None
        
        last = prices.get(s)
        
        rows.append({
            "symbol": s,
            "last": last,
            "signal": (sig or "n/a").upper(),
            "confidence": conf,
            "forecast_pct": f_pct,
            "std_pct": std_pct,
            "z": z,
            "p_up": p_up
        })
        print("✓")
    
    return pd.DataFrame(rows)

# Run the scan
print("🔍 Scanning universe...")
df_scan = summarize_universe(UNIVERSE[:3])  # Start with 3 stocks for testing

# Display results
print("\n📊 Scan Results:")
print(tabulate(df_scan, headers="keys", tablefmt="github", showindex=False))

In [ ]:
# ═══════════════════════════════════════════════════════════
# STEP 5: EXECUTE TRADES (Main trading engine)
# ═══════════════════════════════════════════════════════════

# This is your complete run_once() function from paper_trading.ipynb
# Copy from your notebook or use the existing one

# For demonstration, here's the simplified version:

def run_once_demo():
    """Simplified trading execution for demo"""
    print("🚀 Running trading engine...")
    
    # 1. Check market status
    try:
        clock = trading.get_clock()
        print(f"   Market is {'OPEN' if clock.is_open else 'CLOSED'}")
    except Exception as e:
        print(f"   Could not check market status: {e}")
    
    # 2. Fetch latest prices
    from paper_trading import fetch_latest_prices
    prices = fetch_latest_prices(UNIVERSE[:3])
    print(f"   Fetched prices for {len(prices)} symbols")
    
    # 3. Run signals (you already did this in Step 4)
    # Use the df_scan from previous cell
    buys = df_scan[
        (df_scan['signal'] == 'BUY') & 
        (df_scan['confidence'] >= MIN_CONF)
    ].copy()
    
    print(f"   Found {len(buys)} BUY candidates")
    
    # 4. Position sizing
    if USE_NOTIONAL:
        per_symbol_dollars = NOTIONAL_CAPITAL / max(1, len(buys))
    else:
        account = trading.get_account()
        bp = float(account.buying_power)
        per_symbol_dollars = 0.95 * bp / max(1, len(buys))
    
    # 5. Prepare orders
    plan = []
    for _, row in buys.iterrows():
        sym = row['symbol']
        px = row['last']
        conf = row['confidence']
        
        if px and px > 0:
            qty = max(1, int(per_symbol_dollars // px))
            tp = round(px * (1 + TP_PCT), 2)
            sl = round(px * (1 - SL_PCT), 2)
            plan.append((sym, px, qty, conf, tp, sl))
    
    # 6. Display plan
    if plan:
        print("\n📋 Trading Plan:")
        print(tabulate(
            [(s, f"${p:.2f}", q, f"{c:.0%}", f"${tp:.2f}", f"${sl:.2f}") 
             for s, p, q, c, tp, sl in plan],
            headers=["Symbol", "Price", "Qty", "Conf", "TP", "SL"],
            tablefmt="fancy_grid"
        ))
    else:
        print("   No trades to execute")
        return []
    
    # 7. Execute (or dry run)
    results = []
    for sym, px, qty, conf, tp, sl in plan:
        if DRY_RUN:
            print(f"   [DRY_RUN] Would BUY {qty} {sym} @ ${px:.2f}")
            results.append((sym, "DRY_RUN"))
        else:
            # Real order submission
            order = MarketOrderRequest(
                symbol=sym,
                qty=qty,
                side=OrderSide.BUY,
                time_in_force=TimeInForce.DAY,
                order_class=OrderClass.BRACKET,
                take_profit=TakeProfitRequest(limit_price=tp),
                stop_loss=StopLossRequest(stop_price=sl)
            )
            try:
                res = trading.submit_order(order)
                oid = getattr(res, "id", "unknown")
                print(f"   ✅ Submitted {sym}: Order ID {oid}")
                results.append((sym, oid))
                time.sleep(0.3)
            except Exception as e:
                print(f"   ❌ Failed {sym}: {e}")
    
    return results

# Run it!
trade_results = run_once_demo()

In [ ]:
# ═══════════════════════════════════════════════════════════
# STEP 6: MONITOR POSITIONS
# ═══════════════════════════════════════════════════════════

# Check your current positions
positions = trading.get_all_positions()

if not positions:
    print("📭 No open positions")
else:
    print(f"📊 You have {len(positions)} open position(s):\n")
    
    table_data = []
    for pos in positions:
        # Calculate unrealized P&L %
        entry = float(pos.avg_entry_price)
        current = float(pos.current_price)
        pnl_pct = ((current - entry) / entry) * 100
        
        table_data.append([
            pos.symbol,
            pos.qty,
            f"${entry:.2f}",
            f"${current:.2f}",
            f"${pos.unrealized_pl:.2f}",
            f"{pnl_pct:+.2f}%"
        ])
    
    print(tabulate(
        table_data,
        headers=["Symbol", "Qty", "Entry", "Current", "Unrealized P&L", "%"],
        tablefmt="fancy_grid"
    ))

# Check pending orders
orders = trading.get_orders(filter=GetOrdersRequest(status=QueryOrderStatus.OPEN))

if not orders:
    print("\n📋 No pending orders")
else:
    print(f"\n📋 You have {len(orders)} pending order(s):")
    for order in orders:
        print(f"   {order.symbol}: {order.side} {order.qty} @ {order.order_type}")

---

## 🎓 Summary: What Each Piece Does

### **Core Trading Functions** (your custom code in `trading_functions.py`)

1. **`unified_bayesian_gp_forecast(ticker)`** 🧠
   - **The brain of your system**
   - Combines 3 independent models (Bayesian, GP, RSI)
   - Generates BUY/SELL/HOLD signal with confidence
   - **Use it**: To get a trading recommendation for any stock

2. **`rsi_calculator(df)`** 📊
   - Adds RSI technical indicator to price data
   - **Use it**: As part of feature engineering for models

3. **`bayesian_rsi_signal(rsi_value)`** 🎯
   - Converts RSI into probabilistic signal
   - **Use it**: Inside forecasting functions

4. **`update_beliefs_with_data(...)`** 📈
   - Bayesian linear regression core
   - **Use it**: Called by unified_bayesian_gp_forecast

5. **`gp_predict(...)`** 🌀
   - Gaussian Process prediction engine
   - **Use it**: Called by unified_bayesian_gp_forecast

---

### **Paper Trading Functions** (your automation in `paper_trading.ipynb`)

1. **`fetch_latest_prices(symbols)`** 💰
   - Gets current stock prices from Alpaca
   - **Use it**: Before position sizing or order execution

2. **`extract_signal_confidence(result)`** 🔍
   - Parses model output into (signal, confidence)
   - **Use it**: After running any forecast

3. **`preview_signal(symbol)`** 👀
   - Quick single-stock preview
   - **Use it**: Test individual stocks before batch processing

4. **`summarize_universe(symbols)`** 📊
   - Batch scan all stocks in your watchlist
   - **Use it**: Daily market scan to find opportunities

5. **`run_once()`** ⚙️
   - **The main execution engine**
   - Scans → Filters → Sizes → Executes trades
   - **Use it**: Your primary trading command

---

### **How They Connect in Real Trading**

```
Morning Routine:
1. Run summarize_universe(UNIVERSE) → Find best stocks
2. Review results → Check signals make sense
3. Adjust thresholds if needed → Fine-tune strategy
4. Run run_once() → Execute trades
5. Monitor positions → Check throughout day

If something looks odd:
- Use preview_signal("AAPL") → Deep dive single stock
- Check unified_bayesian_gp_forecast("AAPL") → See full model output
- Adjust MIN_CONF, TP_PCT, SL_PCT → Tune risk parameters
```

---

### **Key Takeaways**

✅ **`unified_bayesian_gp_forecast`** = Your forecasting brain  
✅ **`run_once`** = Your execution engine  
✅ **`summarize_universe`** = Your daily scanner  
✅ **Alpaca API** = Your broker connection  

🎯 **Start with DRY_RUN=True** → Test everything  
🎯 **Use small UNIVERSE** → 3-5 stocks at first  
🎯 **Monitor for 30 days** → Build confidence  
🎯 **Then scale up** → Add more stocks, real capital  

---

## 📖 Glossary: Trading & ML Terms

---

### **Trading Terms**

| Term | Meaning | Example |
|------|---------|---------|
| **Paper Trading** | Simulated trading with fake money (practice) | Your Alpaca account with `paper=True` |
| **Bracket Order** | Order with automatic TP and SL attached | BUY + Take Profit @ +4% + Stop Loss @ -2% |
| **Take Profit (TP)** | Automatic sell when price hits profit target | Entry $100 → TP $104 (+4%) |
| **Stop Loss (SL)** | Automatic sell when price hits loss limit | Entry $100 → SL $98 (-2%) |
| **Long Position** | You own the stock (profit if price rises) | Buy 10 shares AAPL |
| **Buying Power** | Cash available to purchase stocks | Your account balance |
| **Notional** | Dollar value of position | 10 shares @ $100 = $1000 notional |
| **Market Order** | Buy/sell immediately at current price | "BUY 10 AAPL at market" |
| **DAY Order** | Order expires end of trading day if not filled | Order canceled at 4pm ET if not executed |

---

### **Machine Learning Terms**

| Term | Meaning | In Your Code |
|------|---------|--------------|
| **Bayesian** | Uses probabilities + prior beliefs | `update_beliefs_with_data()` |
| **Prior** | Initial assumption before seeing data | Assumes equal chance for BUY/SELL/HOLD |
| **Posterior** | Updated belief after seeing data | Weighted average incorporating new prices |
| **Gaussian Process (GP)** | Non-linear pattern learner | `gp_predict()` - captures complex curves |
| **Ensemble** | Combining multiple models | Weighted average of Bayesian + GP + RSI |
| **Confidence Interval** | Range of likely values | forecast ± 2*std = 95% CI |
| **Z-score** | Statistical edge (how many std deviations) | forecast / std → z=0.5 means moderate edge |
| **Kernel** | Similarity function for GP | RBF kernel - measures how close points are |
| **Forecast** | Predicted future return | 0.012 = +1.2% expected return |
| **Uncertainty (std)** | How confident model is | Low std = confident, high std = uncertain |

---

### **Technical Analysis Terms**

| Term | Meaning | In Your Code |
|------|---------|--------------|
| **RSI (14)** | Relative Strength Index (14-day) | `rsi_calculator()` |
| **Overbought** | RSI > 70 (may drop soon) | `bayesian_rsi_signal()` suggests SELL |
| **Oversold** | RSI < 30 (may rise soon) | `bayesian_rsi_signal()` suggests BUY |
| **Volatility** | How much price fluctuates | `df['Volatility'].rolling(5).std()` |
| **Lagged Returns** | Past returns used as features | `Return_lag1`, `Return_lag2`, etc. |
| **OHLCV** | Open, High, Low, Close, Volume | Standard price data format |

---

### **Your Custom Terminology**

| Your Term | What It Means |
|-----------|---------------|
| **Universe** | List of stocks you scan | `UNIVERSE = ["AAPL", "MSFT", ...]` |
| **Run Once** | Execute one trading cycle | `run_once()` - scan + filter + execute |
| **DRY_RUN** | Preview mode (don't actually trade) | `DRY_RUN = True` |
| **MIN_CONF** | Minimum confidence to trade | 0.65 = require 65% confidence |
| **Ensemble Fallback** | Use statistical edge if no explicit signal | Check Z-score + P(upside) |
| **Relaxed Selection** | Lower standards if no picks | Accept weaker signals to get TARGET_MIN_PICKS |

---

## 🔧 Troubleshooting Common Issues

---

### **Problem: No signals generated**

**Symptoms**: `run_once()` returns empty list

**Fixes**:
```python
# 1. Check if thresholds are too strict
print(f"MIN_CONF: {MIN_CONF}")  # Try lowering to 0.60
print(f"MIN_PROB_UP: {MIN_PROB_UP}")  # Try lowering to 0.55

# 2. Check individual stock signals
for sym in UNIVERSE[:3]:
    result = unified_bayesian_gp_forecast(sym)
    print(f"{sym}: {result['final_signal']} @ {result['final_confidence']:.0%}")

# 3. Enable relaxed selection
RELAX_SELECTION = True
ALT_SELECTION = True
```

---

### **Problem: Orders fail to submit**

**Symptoms**: `APIError` or "insufficient buying power"

**Fixes**:
```python
# 1. Check API credentials
print("API_KEY:", bool(API_KEY))
print("SECRET_KEY:", bool(SECRET_KEY))

# 2. Check account status
account = trading.get_account()
print(f"Status: {account.status}")
print(f"Buying Power: ${account.buying_power}")
print(f"Pattern Day Trader: {account.pattern_day_trader}")

# 3. Use notional mode for testing
USE_NOTIONAL = True
NOTIONAL_CAPITAL = 100

# 4. Reduce position sizes
MAX_ORDERS_PER_RUN = 1  # Start with 1 trade
```

---

### **Problem: Forecasts look unrealistic**

**Symptoms**: Predicting +50% returns or negative confidence

**Fixes**:
```python
# 1. Check data quality
import yfinance as yf
df = yf.download("AAPL", period="200d")
print(df.tail(20))  # Check for gaps, NaN values

# 2. Increase data history
result = unified_bayesian_gp_forecast("AAPL", period="1y")  # More data

# 3. Check for stock splits/dividends
# Yahoo Finance sometimes has issues with adjusted prices
```

---

### **Problem: ImportError for trading_functions**

**Symptoms**: `ModuleNotFoundError: No module named 'functions'`

**Fixes**:
```python
# 1. Check path is correct
import sys
from pathlib import Path
project_root = Path(r"c:\Users\Windows User\My_folder\Finance_project")
print(f"Project root exists: {project_root.exists()}")
print(f"trading_functions.py exists: {(project_root / 'functions' / 'trading_functions.py').exists()}")

# 2. Add to path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# 3. Import with full path
from functions.trading_functions import unified_bayesian_gp_forecast
```

---

### **Problem: Market is closed / orders queued**

**Symptoms**: "Market is CLOSED" message

**This is normal!** 
- DAY orders will execute when market opens (9:30am ET)
- You can still submit orders after hours
- Check status:
```python
clock = trading.get_clock()
print(f"Market open: {clock.is_open}")
print(f"Next open: {clock.next_open}")
print(f"Next close: {clock.next_close}")
```

---

### **Problem: Plots not showing**

**Symptoms**: `unified_bayesian_gp_forecast()` runs but no plots appear

**Fixes**:
```python
# 1. Ensure matplotlib is working
import matplotlib.pyplot as plt
plt.plot([1,2,3])
plt.show()  # Should show a simple plot

# 2. Check if running in suppressed mode
# Remove suppress_plots() wrapper when testing individual stocks

# 3. Use Jupyter magic command
%matplotlib inline  # At top of notebook
```

---

### **Problem: "Not enough data" errors**

**Symptoms**: Functions return `None` or print warnings

**Fixes**:
```python
# 1. Some stocks have limited history
# Try different tickers with longer trading history

# 2. Increase period
result = unified_bayesian_gp_forecast("NEWSTOCK", period="2y")

# 3. Check if ticker is valid
import yfinance as yf
data = yf.download("BADTICKER", period="1d")
print(f"Data exists: {not data.empty}")
```

---

### **Getting Help**

If you're still stuck:

1. **Check API status**: https://status.alpaca.markets/
2. **Review Alpaca docs**: https://docs.alpaca.markets/
3. **Print debug info**:
   ```python
   DEBUG_SELECTION = True  # See filtering logic
   print(result)  # Print full forecast output
   ```

---

---

## 🎯 Quick Start Checklist

Use this to get up and running fast:

### **First Time Setup** ✓

- [ ] Install dependencies: `pip install yfinance alpaca-py pandas numpy scipy matplotlib python-dotenv tabulate`
- [ ] Create `.env` file with Alpaca API keys
- [ ] Test API connection: `trading.get_account()`
- [ ] Run single forecast: `unified_bayesian_gp_forecast("AAPL")`

### **Before Each Trading Session** ✓

- [ ] Set `DRY_RUN = True` for testing
- [ ] Define your `UNIVERSE` (start small: 3-5 stocks)
- [ ] Configure risk: `TP_PCT`, `SL_PCT`, `MIN_CONF`
- [ ] Check market hours: `trading.get_clock().is_open`

### **Daily Trading Workflow** ✓

1. [ ] **Scan**: `df = summarize_universe(UNIVERSE)`
2. [ ] **Review**: Check signals make sense
3. [ ] **Test**: `preview_signal("AAPL")` for deep dive
4. [ ] **Execute**: `run_once()` to place trades
5. [ ] **Monitor**: Check positions throughout day

### **Weekly Maintenance** ✓

- [ ] Review performance (win rate, P&L)
- [ ] Adjust thresholds based on results
- [ ] Add/remove stocks from universe
- [ ] Check for strategy drift (still profitable?)

---

## 🚀 You're Ready to Trade!

**Remember**:
- 🧪 Start with **DRY_RUN=True** and paper trading
- 📊 Use **small universe** (3-5 stocks) initially
- 💰 Test with **NOTIONAL_CAPITAL** before real money
- 📈 **Monitor for 30 days** before going live
- 🎯 **Always use bracket orders** (TP + SL)

**Your main commands**:
```python
# Quick preview
preview_signal("AAPL")

# Full scan
df = summarize_universe(UNIVERSE)

# Execute trades
results = run_once()

# Check positions
positions = trading.get_all_positions()
```

---

### **Next Steps**

1. **Paper trade for 1 month** with `DRY_RUN=False`
2. **Track your results** (win rate, profit factor)
3. **Tune parameters** (MIN_CONF, TP_PCT, SL_PCT)
4. **Scale gradually** (more stocks, more capital)
5. **Consider live trading** only after consistent paper profits

---

**Good luck! 📈💰**

---

---

## 🔬 Advanced Features: Bollinger Bands, Monte Carlo, Adaptive Thresholds

This section addresses advanced questions and adds powerful new capabilities to your trading system.

### **1️⃣ Adding Bollinger Bands to Your Models**

**What are Bollinger Bands?**
- Upper Band = 20-day moving average + (2 × standard deviation)
- Middle Band = 20-day moving average
- Lower Band = 20-day moving average - (2 × standard deviation)

**Trading Logic:**
- Price near lower band → oversold → potential BUY
- Price near upper band → overbought → potential SELL
- Band width = volatility (wide = high volatility, narrow = low volatility)

**How to integrate:**
Add Bollinger Bands as features in `unified_bayesian_gp_forecast()`

In [ ]:
# Add this function to trading_functions.py or run here

def calculate_bollinger_bands(df, period=20, num_std=2):
    """
    Calculate Bollinger Bands and add to DataFrame.
    
    Returns DataFrame with:
    - bb_middle: 20-day moving average
    - bb_upper: middle + 2*std
    - bb_lower: middle - 2*std
    - bb_width: upper - lower (volatility measure)
    - bb_position: where price is relative to bands (0=lower, 0.5=middle, 1=upper)
    """
    df = df.copy()
    
    # Calculate middle band (SMA)
    df['bb_middle'] = df['Close'].rolling(window=period).mean()
    
    # Calculate standard deviation
    rolling_std = df['Close'].rolling(window=period).std()
    
    # Upper and lower bands
    df['bb_upper'] = df['bb_middle'] + (num_std * rolling_std)
    df['bb_lower'] = df['bb_middle'] - (num_std * rolling_std)
    
    # Band width (volatility indicator)
    df['bb_width'] = df['bb_upper'] - df['bb_lower']
    
    # Normalized position (0 to 1, where 0.5 = middle)
    df['bb_position'] = (df['Close'] - df['bb_lower']) / (df['bb_upper'] - df['bb_lower'])
    
    return df

# Example: Test on AAPL
import yfinance as yf
import matplotlib.pyplot as plt

df_test = yf.download("AAPL", period="6mo", interval="1d")
df_test = calculate_bollinger_bands(df_test)

# Plot
plt.figure(figsize=(14, 6))
plt.plot(df_test.index, df_test['Close'], label='Close Price', color='black', linewidth=2)
plt.plot(df_test.index, df_test['bb_upper'], label='Upper Band', color='red', linestyle='--')
plt.plot(df_test.index, df_test['bb_middle'], label='Middle (SMA 20)', color='blue', linestyle='--')
plt.plot(df_test.index, df_test['bb_lower'], label='Lower Band', color='green', linestyle='--')
plt.fill_between(df_test.index, df_test['bb_upper'], df_test['bb_lower'], alpha=0.1, color='gray')
plt.title('AAPL with Bollinger Bands')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Show last values
print(f"\nCurrent Bollinger Band Position: {df_test['bb_position'].iloc[-1]:.2f}")
print(f"0.0-0.3 = Oversold (near lower band)")
print(f"0.7-1.0 = Overbought (near upper band)")
print(f"0.4-0.6 = Neutral (near middle)")

### **2️⃣ Monte Carlo Simulation: "How much money would I make?"**

This answers: **"If I ran my strategy for a month with $100, what would happen?"**

Monte Carlo simulates your strategy thousands of times with randomness to show:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def monte_carlo_strategy_simulation(
    initial_capital=100,
    num_simulations=1000,
    days_per_month=20,
    avg_trades_per_day=1,
    win_rate=0.55,  # 55% win rate
    avg_win=0.04,   # Average win = +4%
    avg_loss=0.02,  # Average loss = -2%
    position_size_pct=0.33  # Use 33% of capital per trade
):
    """
    Monte Carlo simulation of trading strategy.
    
    Args:
        initial_capital: Starting $ (default $100)
        num_simulations: Number of random scenarios to run
        days_per_month: Trading days in a month
        avg_trades_per_day: How many trades per day
        win_rate: Probability of winning trade (0.55 = 55%)
        avg_win: Average winning return (0.04 = 4%)
        avg_loss: Average losing return (0.02 = 2%)
        position_size_pct: Fraction of capital per trade
    
    Returns:
        DataFrame with simulation results
    """
    results = []
    
    for sim in range(num_simulations):
        capital = initial_capital
        daily_capital = [capital]
        
        for day in range(days_per_month):
            # How many trades today?
            num_trades = np.random.poisson(avg_trades_per_day)
            
            for trade in range(num_trades):
                if capital <= 0:
                    break
                
                # Position size
                position_value = capital * position_size_pct
                
                # Win or loss?
                is_win = np.random.random() < win_rate
                
                if is_win:
                    # Random win around avg_win (normal distribution)
                    return_pct = np.random.normal(avg_win, avg_win * 0.3)
                    profit = position_value * return_pct
                else:
                    # Random loss around avg_loss
                    return_pct = -np.random.normal(avg_loss, avg_loss * 0.3)
                    profit = position_value * return_pct
                
                capital += profit
            
            daily_capital.append(capital)
        
        # Store final results
        final_capital = daily_capital[-1]
        total_return = (final_capital - initial_capital) / initial_capital
        max_drawdown = (min(daily_capital) - initial_capital) / initial_capital
        
        results.append({
            'sim_id': sim,
            'final_capital': final_capital,
            'total_return_pct': total_return * 100,
            'max_drawdown_pct': max_drawdown * 100,
            'path': daily_capital
        })
    
    return pd.DataFrame(results)

# Run simulation
print("🎲 Running Monte Carlo Simulation...")
mc_results = monte_carlo_strategy_simulation(
    initial_capital=100,
    num_simulations=1000,
    win_rate=0.55,  # Adjust based on your backtesting
    avg_win=0.04,
    avg_loss=0.02
)

# Analyze results
print("\n📊 Monte Carlo Results (1000 simulations, 1 month):")
print(f"Starting Capital: $100")
print(f"\nFinal Capital:")
print(f"  Best Case (95th percentile): ${mc_results['final_capital'].quantile(0.95):.2f}")
print(f"  Expected (median): ${mc_results['final_capital'].median():.2f}")
print(f"  Worst Case (5th percentile): ${mc_results['final_capital'].quantile(0.05):.2f}")
print(f"\nTotal Return:")
print(f"  Best: {mc_results['total_return_pct'].quantile(0.95):.1f}%")
print(f"  Expected: {mc_results['total_return_pct'].median():.1f}%")
print(f"  Worst: {mc_results['total_return_pct'].quantile(0.05):.1f}%")
print(f"\nRisk Metrics:")
print(f"  Probability of profit: {(mc_results['final_capital'] > 100).mean():.1%}")
print(f"  Probability of >10% gain: {(mc_results['total_return_pct'] > 10).mean():.1%}")
print(f"  Probability of >10% loss: {(mc_results['total_return_pct'] < -10).mean():.1%}")

# Plot distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Final capital histogram
axes[0, 0].hist(mc_results['final_capital'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(100, color='red', linestyle='--', label='Break-even')
axes[0, 0].axvline(mc_results['final_capital'].median(), color='green', linestyle='--', label='Median')
axes[0, 0].set_xlabel('Final Capital ($)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of Final Capital')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Return distribution
axes[0, 1].hist(mc_results['total_return_pct'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[0, 1].axvline(0, color='red', linestyle='--', label='Break-even')
axes[0, 1].axvline(mc_results['total_return_pct'].median(), color='green', linestyle='--', label='Median')
axes[0, 1].set_xlabel('Total Return (%)')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Returns')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. Sample paths (first 100 simulations)
for i in range(min(100, len(mc_results))):
    path = mc_results.iloc[i]['path']
    axes[1, 0].plot(path, alpha=0.1, color='blue')
axes[1, 0].axhline(100, color='red', linestyle='--', label='Initial Capital')
axes[1, 0].set_xlabel('Day')
axes[1, 0].set_ylabel('Capital ($)')
axes[1, 0].set_title('Sample Capital Paths (100 simulations)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# 4. Max drawdown distribution
axes[1, 1].hist(mc_results['max_drawdown_pct'], bins=50, edgecolor='black', alpha=0.7, color='red')
axes[1, 1].axvline(mc_results['max_drawdown_pct'].median(), color='green', linestyle='--', label='Median')
axes[1, 1].set_xlabel('Max Drawdown (%)')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].set_title('Distribution of Maximum Drawdown')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n💡 Interpretation:")
print(f"With your strategy parameters (55% win rate, +4%/-2% avg), over 1 month:")
print(f"- You have a {(mc_results['final_capital'] > 100).mean():.0%} chance of making money")
print(f"- Expected to end with ${mc_results['final_capital'].median():.2f}")
print(f"- Worst case (5%): lose {abs(mc_results['total_return_pct'].quantile(0.05)):.1f}%")
print(f"- Best case (95%): gain {mc_results['total_return_pct'].quantile(0.95):.1f}%")

### **3️⃣ Adaptive Thresholds: Bayesian Parameter Tuning**

Instead of fixed `MIN_CONF = 0.65`, make it adapt based on recent performance!

In [ ]:
def adaptive_threshold_calculator(trade_history, initial_min_conf=0.65):
    """
    Calculate adaptive MIN_CONF based on recent trading performance.
    
    Logic:
    - If win rate is high → lower MIN_CONF (take more trades)
    - If win rate is low → raise MIN_CONF (be more selective)
    - Use Bayesian updating to smooth adjustments
    
    Args:
        trade_history: List of dicts with keys: 'confidence', 'profit' (positive or negative)
        initial_min_conf: Starting threshold
    
    Returns:
        new_min_conf: Adjusted threshold
    """
    if not trade_history or len(trade_history) < 5:
        return initial_min_conf
    
    # Convert to DataFrame for analysis
    df = pd.DataFrame(trade_history)
    
    # Calculate recent win rate (last 20 trades)
    recent = df.tail(20)
    win_rate = (recent['profit'] > 0).mean()
    
    # Calculate profit factor (total wins / total losses)
    wins = recent[recent['profit'] > 0]['profit'].sum()
    losses = abs(recent[recent['profit'] < 0]['profit'].sum())
    profit_factor = wins / losses if losses > 0 else 2.0
    
    # Bayesian adjustment
    # Prior: MIN_CONF = 0.65, uncertainty = 0.1
    prior_mean = initial_min_conf
    prior_std = 0.1
    
    # Likelihood based on performance
    if win_rate >= 0.60 and profit_factor >= 1.5:
        # Good performance → lower threshold
        suggested_conf = prior_mean - 0.05
    elif win_rate <= 0.45 or profit_factor <= 1.0:
        # Poor performance → raise threshold
        suggested_conf = prior_mean + 0.10
    else:
        # Neutral performance → small adjustment
        suggested_conf = prior_mean + (0.55 - win_rate) * 0.2
    
    # Bayesian posterior (weighted average of prior and suggestion)
    data_weight = min(len(recent) / 20.0, 1.0)  # More data = more weight
    posterior_mean = (1 - data_weight) * prior_mean + data_weight * suggested_conf
    
    # Clamp to reasonable range [0.55, 0.80]
    new_min_conf = np.clip(posterior_mean, 0.55, 0.80)
    
    print(f"📊 Adaptive Threshold Analysis:")
    print(f"   Recent win rate: {win_rate:.1%}")
    print(f"   Profit factor: {profit_factor:.2f}")
    print(f"   Old MIN_CONF: {initial_min_conf:.2f}")
    print(f"   New MIN_CONF: {new_min_conf:.2f}")
    print(f"   → {'LOWERED' if new_min_conf < initial_min_conf else 'RAISED'} threshold")
    
    return new_min_conf

# Example usage
example_trade_history = [
    {'confidence': 0.70, 'profit': 3.50},   # Win
    {'confidence': 0.68, 'profit': -1.80},  # Loss
    {'confidence': 0.75, 'profit': 4.20},   # Win
    {'confidence': 0.72, 'profit': 3.80},   # Win
    {'confidence': 0.65, 'profit': -2.00},  # Loss
    {'confidence': 0.80, 'profit': 5.00},   # Win
    {'confidence': 0.71, 'profit': 3.20},   # Win
    {'confidence': 0.69, 'profit': -1.50},  # Loss
    {'confidence': 0.77, 'profit': 4.50},   # Win
    {'confidence': 0.73, 'profit': 3.90},   # Win
]

new_threshold = adaptive_threshold_calculator(example_trade_history, initial_min_conf=0.65)
print(f"\n✅ Use this in run_once(): MIN_CONF = {new_threshold:.2f}")

### **4️⃣ Trading Assistant: Code That Tells You What To Do**

This analyzes your trading performance and explicitly tells you what action to take next.

In [ ]:
def trading_assistant(trade_history, current_positions, account_summary):
    """
    Analyzes your trading system and tells you EXACTLY what to do next.
    
    Args:
        trade_history: List of past trades with profit/loss
        current_positions: List of open positions
        account_summary: Dict with account info (portfolio_value, cash, etc.)
    
    Returns:
        List of recommendations with priority and reasoning
    """
    recommendations = []
    
    if not trade_history or len(trade_history) < 10:
        recommendations.append({
            'priority': 'HIGH',
            'action': 'RUN_ONCE',
            'reason': 'Not enough trading history. Execute more trades to build data.',
            'details': f'Only {len(trade_history)} trades recorded. Need 20+ for reliable analysis.'
        })
        return recommendations
    
    # Convert to DataFrame
    df = pd.DataFrame(trade_history)
    
    # Calculate metrics
    total_trades = len(df)
    wins = (df['profit'] > 0).sum()
    losses = (df['profit'] < 0).sum()
    win_rate = wins / total_trades if total_trades > 0 else 0
    
    total_profit = df[df['profit'] > 0]['profit'].sum()
    total_loss = abs(df[df['profit'] < 0]['profit'].sum())
    profit_factor = total_profit / total_loss if total_loss > 0 else 0
    
    avg_win = df[df['profit'] > 0]['profit'].mean() if wins > 0 else 0
    avg_loss = abs(df[df['profit'] < 0]['profit'].mean()) if losses > 0 else 0
    
    # Calculate drawdown
    cumulative = (df['profit'].cumsum() + account_summary.get('cash', 100))
    running_max = cumulative.expanding().max()
    drawdown = ((cumulative - running_max) / running_max * 100).min()
    
    # Expected value per trade
    ev_per_trade = (win_rate * avg_win) - ((1 - win_rate) * avg_loss)
    
    # Sharpe ratio (simplified)
    returns = df['profit'] / account_summary.get('portfolio_value', 100)
    sharpe = returns.mean() / returns.std() if returns.std() > 0 else 0
    
    print("="*70)
    print("🤖 TRADING ASSISTANT ANALYSIS")
    print("="*70)
    print(f"\n📊 Performance Metrics:")
    print(f"   Total Trades: {total_trades}")
    print(f"   Win Rate: {win_rate:.1%} ({wins}W / {losses}L)")
    print(f"   Profit Factor: {profit_factor:.2f}")
    print(f"   Avg Win: ${avg_win:.2f}")
    print(f"   Avg Loss: ${avg_loss:.2f}")
    print(f"   Expected Value/Trade: ${ev_per_trade:.2f}")
    print(f"   Max Drawdown: {drawdown:.1f}%")
    print(f"   Sharpe Ratio: {sharpe:.2f}")
    
    # DECISION LOGIC
    
    # 1. Check if strategy is losing money
    if ev_per_trade < 0 or profit_factor < 1.0:
        recommendations.append({
            'priority': 'CRITICAL',
            'action': 'TUNE_THRESHOLDS',
            'reason': 'Strategy is losing money on average.',
            'details': f'EV/trade = ${ev_per_trade:.2f}, Profit Factor = {profit_factor:.2f}. ' +
                       'Increase MIN_CONF to 0.75+ or widen TP/SL ratio.'
        })
    
    # 2. Check win rate
    if win_rate < 0.45:
        recommendations.append({
            'priority': 'HIGH',
            'action': 'INCREASE_MIN_CONF',
            'reason': f'Win rate too low ({win_rate:.1%}).',
            'details': f'Current win rate {win_rate:.1%} < 45%. Raise MIN_CONF by 0.10 to filter weaker signals.'
        })
    elif win_rate > 0.70:
        recommendations.append({
            'priority': 'MEDIUM',
            'action': 'DECREASE_MIN_CONF',
            'reason': f'Win rate very high ({win_rate:.1%}) - may be too conservative.',
            'details': f'You can afford to lower MIN_CONF by 0.05 to capture more opportunities.'
        })
    
    # 3. Check drawdown
    if drawdown < -15:
        recommendations.append({
            'priority': 'CRITICAL',
            'action': 'REDUCE_POSITION_SIZE',
            'reason': f'Excessive drawdown ({drawdown:.1f}%).',
            'details': 'Cut position sizes in half or tighten stop losses to preserve capital.'
        })
    
    # 4. Check if not trading enough
    if total_trades < 20:
        recommendations.append({
            'priority': 'MEDIUM',
            'action': 'EXPAND_UNIVERSE',
            'reason': 'Low trade frequency.',
            'details': f'Only {total_trades} trades. Add more stocks to UNIVERSE or lower MIN_CONF slightly.'
        })
    
    # 5. Check concentration risk
    if current_positions:
        num_positions = len(current_positions)
        if num_positions > 10:
            recommendations.append({
                'priority': 'MEDIUM',
                'action': 'REDUCE_MAX_ORDERS',
                'reason': f'Too many positions ({num_positions}).',
                'details': 'Lower MAX_ORDERS_PER_RUN to focus on best opportunities.'
            })
        elif num_positions < 3 and total_trades > 20:
            recommendations.append({
                'priority': 'LOW',
                'action': 'INCREASE_MAX_ORDERS',
                'reason': 'Under-diversified portfolio.',
                'details': f'Only {num_positions} positions. Consider MAX_ORDERS_PER_RUN = 5-8.'
            })
    
    # 6. Check if profitable but can optimize
    if ev_per_trade > 0 and 0.50 <= win_rate <= 0.65 and profit_factor > 1.3:
        recommendations.append({
            'priority': 'LOW',
            'action': 'RUN_MONTE_CARLO',
            'reason': 'Strategy looks profitable! Quantify expected returns.',
            'details': 'Run Monte Carlo simulation to estimate monthly P&L distribution.'
        })
    
    # 7. Suggest backtesting if never done
    if total_trades < 30:
        recommendations.append({
            'priority': 'MEDIUM',
            'action': 'BACKTEST_STRATEGY',
            'reason': 'Need historical validation.',
            'details': 'Test strategy on 6-12 months of historical data before committing more capital.'
        })
    
    # 8. Good performance - stay the course
    if ev_per_trade > 1.0 and win_rate > 0.52 and profit_factor > 1.5 and drawdown > -10:
        recommendations.append({
            'priority': 'LOW',
            'action': 'CONTINUE_CURRENT_STRATEGY',
            'reason': '✅ Strategy performing well!',
            'details': f'Win rate {win_rate:.1%}, PF {profit_factor:.2f}, EV ${ev_per_trade:.2f}/trade. Keep executing.'
        })
    
    # Sort by priority
    priority_order = {'CRITICAL': 0, 'HIGH': 1, 'MEDIUM': 2, 'LOW': 3}
    recommendations.sort(key=lambda x: priority_order.get(x['priority'], 99))
    
    # Print recommendations
    print(f"\n🎯 RECOMMENDED ACTIONS (in priority order):")
    print("="*70)
    
    for i, rec in enumerate(recommendations, 1):
        priority_color = {
            'CRITICAL': '🔴',
            'HIGH': '🟠',
            'MEDIUM': '🟡',
            'LOW': '🟢'
        }.get(rec['priority'], '⚪')
        
        print(f"\n{i}. {priority_color} [{rec['priority']}] {rec['action']}")
        print(f"   Reason: {rec['reason']}")
        print(f"   Details: {rec['details']}")
    
    print("\n" + "="*70)
    
    return recommendations

# Example usage
example_trades = [
    {'confidence': 0.70, 'profit': 3.50},
    {'confidence': 0.68, 'profit': -1.80},
    {'confidence': 0.75, 'profit': 4.20},
    {'confidence': 0.72, 'profit': -2.00},
    {'confidence': 0.65, 'profit': -1.50},
    {'confidence': 0.80, 'profit': 5.00},
    {'confidence': 0.71, 'profit': -2.20},
    {'confidence': 0.69, 'profit': -1.80},
    {'confidence': 0.77, 'profit': 4.50},
    {'confidence': 0.73, 'profit': -1.90},
    {'confidence': 0.76, 'profit': 3.80},
    {'confidence': 0.68, 'profit': -2.10},
]

example_positions = [
    {'symbol': 'AAPL', 'qty': 5, 'entry': 180.00},
    {'symbol': 'MSFT', 'qty': 3, 'entry': 370.00},
]

example_account = {
    'cash': 50.00,
    'portfolio_value': 105.50
}

# Run the assistant
recommendations = trading_assistant(example_trades, example_positions, example_account)

---

## 📋 **How To Use Everything Together**

Here's the **complete workflow** combining all the advanced features:

### **Step 1: Initial Setup**
```python
# Define your universe
UNIVERSE = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'META', 'NVDA']
MIN_CONF = 0.65
```

### **Step 2: Run the Strategy**
```python
# Execute once
run_once(dry_run=False)  # Real trades on paper account
```

### **Step 3: Analyze with Bollinger Bands**
```python
# Check if signals align with technical analysis
df = yf.download('AAPL', period='3mo')
bb_df = calculate_bollinger_bands(df, window=20, num_std=2)

# Last row shows current position relative to bands
print(bb_df.tail(3))
```

### **Step 4: Check Performance with Monte Carlo**
```python
# Estimate expected returns over next month
results = monte_carlo_strategy_simulation(
    initial_capital=100,
    avg_trades_per_day=2,
    win_rate=0.55,
    avg_win_pct=4.0,
    avg_loss_pct=2.0,
    days=30,
    num_simulations=1000
)

# See probability of profit
print(f"Probability of profit: {results['prob_profit']:.1%}")
```

### **Step 5: Optimize with Adaptive Thresholds**
```python
# After 30+ trades, tune MIN_CONF dynamically
trade_history = [
    {'confidence': 0.70, 'profit': 3.50},
    {'confidence': 0.68, 'profit': -1.80},
    # ... more trades
]

new_conf = adaptive_threshold_calculator(trade_history, initial_min_conf=0.65)
print(f"New MIN_CONF: {new_conf:.2f}")
```

### **Step 6: Get Explicit Guidance from Trading Assistant**
```python
# Let the assistant tell you what to do
recommendations = trading_assistant(
    trade_history=trade_history,
    current_positions=get_current_positions(),
    account_summary={'cash': 50, 'portfolio_value': 105}
)

# Follow the top priority recommendations
for rec in recommendations[:3]:  # Top 3 actions
    print(f"{rec['action']}: {rec['details']}")
```

---

### **🔄 Daily Workflow Example**

```python
# Morning routine
# 1. Check what the assistant says
recommendations = trading_assistant(my_trades, my_positions, my_account)

# 2. If assistant says "CONTINUE", run strategy
if recommendations[0]['action'] == 'CONTINUE_CURRENT_STRATEGY':
    run_once(dry_run=False)

# 3. If assistant says "TUNE_THRESHOLDS", adjust
elif recommendations[0]['action'] == 'TUNE_THRESHOLDS':
    MIN_CONF = adaptive_threshold_calculator(my_trades)
    run_once(dry_run=False)

# 4. If assistant says "BACKTEST", run Monte Carlo
elif recommendations[0]['action'] == 'RUN_MONTE_CARLO':
    monte_carlo_strategy_simulation(...)
```

---

This creates a **fully automated, self-optimizing trading system** that:
- ✅ Scans stocks with Bayesian forecasting
- ✅ Validates signals with Bollinger Bands
- ✅ Quantifies risk with Monte Carlo
- ✅ Tunes itself with adaptive thresholds
- ✅ **Tells you exactly what to do** with Trading Assistant

**You now have a complete AI trading co-pilot! 🚀**